In [1]:
import os
import torch  # PyTorch
from monai.losses import DiceLoss  # MONAI loss function
from monai.optimizers import Novograd  # MONAI optimizer
from monai.networks.nets import UNet  # MONAI UNet model
from monai.transforms import (
    Compose,
    LoadImage,
    EnsureChannelFirst,
    ScaleIntensity,
    Resize,
    ToTensor,
    Lambda,
)
from monai.data import Dataset, DataLoader

# Define paths
data_dir = "archive/ultrasound breast classification/train"  # Update this path
benign_dir = os.path.join(data_dir, "benign")
malignant_dir = os.path.join(data_dir, "malignant")

# Define transformations


def rgb_to_grayscale(image):
    if image.shape[0] == 3:  # If it's an RGB image
        return torch.mean(image, dim=0, keepdim=True)
    return image  # If it's already grayscale, return as is


def create_label_mask(label, shape):
    # Create a one-hot encoded mask
    mask = torch.zeros((2, shape[0], shape[1]))
    mask[label, :, :] = 1
    return mask


# Define image-only transformations
image_transforms = Compose(
    [
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        ScaleIntensity(),
        Resize((224, 224)),
        Lambda(rgb_to_grayscale),
        ToTensor(),
    ]
)

# Define the full transform that applies image transforms only to 'img' key
transform = Compose(
    [
        Lambda(
            lambda x: {
                "img": image_transforms(x["img"]),
                "label": create_label_mask(x["label"], (224, 224)),
            }
        )
    ]
)
# Prepare the dataset
images = []
labels = []

# Load benign images
for filename in os.listdir(benign_dir):
    if filename.endswith((".png", ".jpg", ".jpeg")):  # Add more formats if needed
        images.append(os.path.join(benign_dir, filename))
        labels.append(0)  # Label 0 for benign

# Load malignant images
for filename in os.listdir(malignant_dir):
    if filename.endswith((".png", ".jpg", ".jpeg")):  # Add more formats if needed
        images.append(os.path.join(malignant_dir, filename))
        labels.append(1)  # Label 1 for malignant

# Create a dataset
dataset = Dataset(
    data=[{"img": img, "label": label} for img, label in zip(images, labels)],
    transform=transform,
)

# Create a DataLoader
batch_size = 64  # Define your batch size
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for batch_data in data_loader:
    # images, labels = batch_data['img'].to(device), batch_data['label'].to(device)
    print(batch_data["img"][0].shape)
    break

/home/haiche/anaconda3/envs/haiche/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1, 224, 224])


In [2]:
model = UNet(
    spatial_dims=2,  # Change to 3 if you have 3D data
    in_channels=1,  # Change according to your data
    out_channels=2,  # Change according to your number of classes
    channels=(16, 32, 64, 128),
    strides=(2, 2, 2),
    num_res_units=2,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_function = DiceLoss(sigmoid=True)
optimizer = Novograd(model.parameters(), lr=1e-4)

# Define the number of epochs
num_epochs = 10  # You can change this to any number you prefer

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_data in data_loader:
        images, labels = batch_data['img'].to(device), batch_data['label'].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(data_loader)}")

Epoch 1/10, Loss: 0.6334448141210219
Epoch 2/10, Loss: 0.6096904324550255
Epoch 3/10, Loss: 0.5868818251525655
Epoch 4/10, Loss: 0.5663891043148789


KeyboardInterrupt: 

In [1]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder  # or your custom dataset

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
data_dir = 'archive/ultrasound breast classification/train/benign'  # Update this path
dataset = ImageFolder(root=data_dir, transform=transform)